# 1. Importing all libraries and creating your Team object.

In [2]:
import sys
import os

import math
import time
import numpy as np
from PIL import Image
import pathlib

#import pynq

from multiprocessing import Process, Pipe, Queue, Event, Manager, Array
img_num = 10000

import cv2
import ipywidgets as widgets
from IPython.display import display

In [4]:
frame_in_w = 640
frame_in_h = 360
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FPS, 25);
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("capture device is open: " + str(videoIn.isOpened()))
imgbox = widgets.Image(format='jpg', height=frame_in_h*2, width=frame_in_w*2)
display(imgbox)

#print(videoIn.get(cv2.CAP_PROP_FPS),videoIn.get(cv2.CAP_PROP_FRAME_WIDTH),videoIn.get(cv2.CAP_PROP_FRAME_HEIGHT))

capture device is open: True


Image(value=b'', format='jpg', height='720', width='1280')

In [ ]:
import os
import cv2
import ipywidgets as widgets
from IPython.display import display

imgbox = widgets.Image(format='jpg', height=300, width=400)
display(imgbox)
root = "D:/yaoyaokong/first_grade_of_graduate/Ultra96/boat2"
for file in os.listdir(root):
    canvas = cv2.imread(os.path.join(root, file))
    imgbox.value = cv2.imencode('.jpg', canvas)[1].tobytes()


Image(value=b'', format='jpg', height='300', width='400')

# 2. Downloading the overlay, perform any any one-time configuration

In [10]:
import numpy as np
frame_in_w = 640
frame_in_h = 360
globalimage = [np.zeros((frame_in_w,frame_in_h,3), np.uint8)for i in range(2)]
globalimage[0] == globalimage[0][:,:,0:3]

array([[[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        ...,
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        ...,
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        ...,
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       ...,

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        ...,
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        ...,
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,

In [ ]:
print('\n**** Running MiaoNet****')

xlnk = pynq.Xlnk()
xlnk.xlnk_reset()

globalimage = np.zeros((frame_in_w,frame_in_h,3), np.uint8)
globalimage_out = np.zeros((frame_in_w,frame_in_h,3), np.uint8)

################### Download the overlay
overlay = pynq.Overlay('dac_sdc.bit')
print("Bitstream loaded")

########## Allocate memory for weights and off-chip buffers
buffer_size = 8
cache_flag = 0
weight_and_bias = xlnk.cma_array(shape=(32310*8), dtype=np.int16, cacheable = cache_flag)
ddr_feature_map_buffer_ping1 = xlnk.cma_array(shape=(4*320*160*8), dtype=np.int16, cacheable = cache_flag)
ddr_feature_map_buffer_pong1 = xlnk.cma_array(shape=(4*320*160*8), dtype=np.int16, cacheable = cache_flag)
ddr_feature_map_reorg1 = xlnk.cma_array(shape=(120*20*40*8), dtype=np.int16, cacheable = cache_flag)
image_pads = [xlnk.cma_array(shape=(160,320,8), dtype=np.int16, cacheable = cache_flag) for i in range(buffer_size)]
outputs = [xlnk.cma_array(shape=(2,20,40,8), dtype=np.int16, cacheable = cache_flag) for i in range(buffer_size)]
print("Allocating memory done")

########### Load parameters from SD card to DDR
params = np.fromfile("SJTU_microe.bin", dtype=np.int16)
np.copyto(weight_and_bias, params.reshape(weight_and_bias.shape))
print("Parameters loading done")



################## Utility functions 
def img_put(image_queue0):
    # preprocess for even number images
    for index in range(0, img_num, 1):
        image_queue0.put(videoIn.read()[1])
        if image_queue0.qsize() > 2:
            image_queue0.get()
        else:
            time.sleep(0.01)

def preprocess(image_queue0,image_queue1,image_out):
    # preprocess for odd number images
    for index in range(0, img_num, 1):
        globalimage = image_queue0.get()
        resized = cv2.resize(globalimage, (320,160))
        converted = cv2.cvtColor(resized, cv2.COLOR_RGB2BGR)
        np.copyto(image_pads[index%buffer_size][:,:,0:3], converted)
        image_queue1.put(image_pads[index%buffer_size].physical_address)
        image_out.put(globalimage)

def postcal(output_queue, result_rectangle):
    
    while output_queue.empty():
        continue
    for i in range(0, img_num):    
        last_conv_out_pad_quant = outputs[output_queue.get()]
        new_out = last_conv_out_pad_quant * 2**-8
        new_out = new_out.transpose([0,3,1,2]).reshape([-1,20,40])[0:10].reshape(2,5,20,40).transpose(0,2,3,1)
        ianchor, icol, irow = np.unravel_index(new_out[:,:,:,4].argmax(), new_out[:,:,:,4].shape)
        if ianchor:
            anchor_w, anchor_h = [4.0113013115312155,5.760873975661669]
        else:
            anchor_w, anchor_h = [1.4940052559648322, 2.3598481287086823]
        obox = new_out[ianchor, icol, irow, 0:4]

#         x = (sigmoid(obox[0]) + irow) * 8 * 640/320
#         y = (sigmoid(obox[1]) + icol) * 8 * 360/160
#         w = np.exp(obox[2]) * anchor_w * 8 * 640/320
#         h = np.exp(obox[3]) * anchor_h * 8 * 360/160
        x = (sigmoid(obox[0]) + irow) * 8 * frame_in_w/320
        y = (sigmoid(obox[1]) + icol) * 8 * frame_in_h/160
        w = np.exp(obox[2]) * anchor_w * 8 * frame_in_w/320
        h = np.exp(obox[3]) * anchor_h * 8 * frame_in_h/160
            
        x1 = int((x - w/2))
        y1 = int((y - h/2))
        x2 = int((x + w/2))
        y2 = int((y + h/2))
        result_rectangle.put([x1,x2,y1,y2])
        
        

def sigmoid(x):
    return 1/(1+np.exp(-x))


################# one-time configuration ##############
MiaoNet = overlay.MiaoDetDAC_0
MiaoNet.write(0x10, weight_and_bias.physical_address)
MiaoNet.write(0x18, ddr_feature_map_buffer_ping1.physical_address)
MiaoNet.write(0x20, ddr_feature_map_buffer_ping1.physical_address)
MiaoNet.write(0x28, ddr_feature_map_buffer_pong1.physical_address)
MiaoNet.write(0x30, ddr_feature_map_buffer_pong1.physical_address)
MiaoNet.write(0x38, ddr_feature_map_reorg1.physical_address)
MiaoNet.write(0x40, ddr_feature_map_reorg1.physical_address)

################# Declare Variable ##############
image_queue_size = int(buffer_size/2)-1
image_queue0 = Queue(image_queue_size)
image_queue1 = Queue(image_queue_size)
image_out = Queue(image_queue_size)
output_queue = Queue(buffer_size-1)
result_rectangle = Queue(2)
rails = pynq.get_rails()
p1 = Process(target=img_put, args=[image_queue0])
p2 = Process(target=postcal, args=(output_queue, result_rectangle))
p3 = Process(target=preprocess, args=(image_queue0, image_queue1,image_out))


**** Running MiaoNet****
Bitstream loaded
Allocating memory done
Parameters loading done


# 3. Processing all images.

The number of CPU is:4
child   p.name:Process-17	p.id10132
child   p.name:Process-18	p.id11516
child   p.name:Process-16	p.id3704
END!!!!!!!!!!!!!!!!!


In [9]:
jupyter nbextension enable --py widgetsnbextension

SyntaxError: invalid syntax (<ipython-input-9-98201c4d83e5>, line 1)

In [1]:
import numpy as np
import cv2
cap = cv2.VideoCapture(0)
while(True):
	#capture frame-by-frame
    ret , frame = cap.read()
    
    #our operation on the frame come here
    gray = cv2.cvtColor(frame , cv2.COLOR_BGR2GRAY)
    
    #display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) &0xFF ==ord('q'):  #按q键退出
    	break
#when everything done , release the capture
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
################### Start to detect ################

print("\n**** Start to detect****")
start = time.time()
if True:
    p1.start()
    p2.start()
    p3.start()
    for index in range(0, img_num, 1):
        # process for even number images
        MiaoNet.write(0x58, outputs[index%buffer_size].physical_address) # tell PL which output buffer to write
        input_physical_address = image_queue1.get() # get physical address of input buffer from preprocess thread
        MiaoNet.write(0x48, input_physical_address) # tell PL which input buffer to read
        MiaoNet.write(0x50, input_physical_address) # tell PL which input buffer to read
        MiaoNet.write(0x00, 1)  # tell PL to start working
        isready = MiaoNet.read(0x00)
        
        while( isready == 1 ): # wait for PL to finish
            isready = MiaoNet.read(0x00)
        output_queue.put(index%buffer_size) # tell process-p2 that another image is dealt with
        
        a = result_rectangle.get() #[x1,x2,y1,y2]
        globalimage = image_out.get()
        globalimage_out = cv2.rectangle(globalimage,(a[0],a[2]),(a[1],a[3]),(0,0,255),3)
        imgbox.value = cv2.imencode('.jpg', globalimage_out)[1].tobytes()
#         # process for odd number images
#         MiaoNet.write(0x58, outputs[(index+1)%buffer_size].physical_address)
#         input_physical_address = image_queue1.get()
#         MiaoNet.write(0x48, input_physical_address) 
#         MiaoNet.write(0x50, input_physical_address) 
#         MiaoNet.write(0x00, 1)  # tell PL to start working
#         isready = MiaoNet.read(0x00)
#         while( isready == 1 ): # wait for PL to finish
#             isready = MiaoNet.read(0x00)
#         output_queue.put((index+1)%buffer_size) # tell process-p2 that another image is dealt with
    
    p1.join()
    p2.join()
    p3.join()
        
end = time.time()
total_time = end - start
print('Processing time: {} seconds.'.format(total_time))

print("**** Detection finished****\n")



**** Start to detect****


# 4. Writing results to XML and any other cleanup.

In [ ]:
xlnk.xlnk_reset()

In [ ]:
#display(videoIn.read()[1])
#import matplotlib.pyplot as plt # plt 用于显示图片
#import matplotlib.image as mpimg # mpimg 用于读取图片
#import numpy as np
#imgbox.value = cv2.imencode('.jpg', videoIn.read()[1])[1].tobytes()
frame_in_w = 640
frame_in_h = 360
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FPS, 20);
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("capture device is open: " + str(videoIn.isOpened()))
imgbox = widgets.Image(format='jpg', height=frame_in_h*2, width=frame_in_w*2)
display(imgbox)
print(videoIn.get(cv2.CAP_PROP_FPS),videoIn.get(cv2.CAP_PROP_FRAME_WIDTH),videoIn.get(cv2.CAP_PROP_FRAME_HEIGHT))


In [ ]:

################## Utility functions 
def img_put(image_queue0):
    # preprocess for even number images
    for index in range(0, img_num, 1):
        image_queue0.put(videoIn.read()[1])
        if image_queue0.qsize() > 2:
            image_queue0.get()
        else:
            time.sleep(0.01)

def preprocess(image_queue0,image_queue1,image_out):
    # preprocess for odd number images
    for index in range(0, img_num, 1):
        globalimage = image_queue0.get()
        resized = cv2.resize(globalimage, (320,160))
        converted = cv2.cvtColor(resized, cv2.COLOR_RGB2BGR)
        np.copyto(image_pads[index%buffer_size][:,:,0:3], converted)
        image_queue1.put(image_pads[index%buffer_size].physical_address)
        image_out.put(globalimage)